In [1]:
import pandas as pd
import json
import geopandas as gpd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import re
import seaborn as sns

# Data

In [4]:
df = pd.read_csv("df_data_suburbs.csv")
df.head()

,name,vacancy_rate,rental_stock,population,rental_pop,region,state
0,Braidwood,0.55,5,1647.0,21.54,Queanbeyan-Palerang Regional,New South Wales
1,Karabar,0.83,29,8241.0,26.85,Queanbeyan-Palerang Regional,New South Wales
2,Queanbeyan,1.41,48,6236.0,38.85,Queanbeyan-Palerang Regional,New South Wales
3,Queanbeyan West,1.24,17,3136.0,29.31,Queanbeyan-Palerang Regional,New South Wales
4,Jerrabomberra,0.63,21,9505.0,19.90,Queanbeyan-Palerang Regional,New South Wales


In [ ]:
states = list(df["state"].unique()[:-1])

def remove_outliers(df):
    """Filter outliers."""
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1
    upper_thresh = q3 + 1.5*iqr
    
    return df[df < upper_thresh]

def create_distplot_by_state(stat):
    stat_by_state = {state: remove_outliers(df[stat][df["state"] == state].dropna()) for state in states}
    
    # group data together
    hist_data = list(stat_by_state.values())
    group_labels = list(stat_by_state.keys())

    # create distplot with custom bin_size
    fig = ff.create_distplot(hist_data, group_labels, show_hist=False, bin_size=0.1)
    fig.update_layout(
        title=f"Suburb {stat.replace('_', ' ').title()} Distribution by State", 
        template="seaborn"
    )
    fig.show()

In [ ]:
stats = ["vacancy_rate", "rental_stock", "population", "rental_pop"]

In [ ]:
create_distplot_by_state("rental_pop")

In [ ]:
df_filtered = df[df["region"] == "Monash"]
df_filtered = df[df["state"] == "Victoria"]

fig = px.scatter(
    df_filtered, 
    x="vacancy_rate", 
    y="rental_stock",
    color="state",
    hover_data=['name'],
    template="seaborn"
)
fig.update_layout(
    xaxis_range=[-0.5, 10],
    yaxis_range=[-2, 100],
)

# Table

In [13]:
df_houses = pd.read_csv("df_tables_houses.csv")
df_town_houses = pd.read_csv("df_tables_town_houses.csv")
df_units = pd.read_csv("df_tables_units.csv")

In [16]:
def merge_suburb(df_houses, state):
    suburbs = df["name"][df["state"] == state].unique()
    df_filtered = df_houses[df_houses["Suburb"].isin(suburbs)].merge(df, left_on='Suburb', right_on='name')
    return df_filtered

In [23]:
df_houses

,Suburb,Median listing price,Median price change - last quarter (%),Median price change - 1 year (%),Median price change - 2 years (%),Median weekly rent,Median yield %,Median rent change - 1 year (%),Previous month sales,Stock on market previous month,Stock variance vs. last year (%),Average days on market
0,Braidwood,789000.0,-1.26,-1.26,13.52,520.0,3.42,4.00,1.0,34.0,30.77,169.0
1,Karabar,750000.0,-1.19,-1.64,-5.90,640.0,4.43,-1.54,4.0,27.0,3.85,67.0
2,Queanbeyan,837000.0,3.65,11.60,2.19,600.0,3.72,3.44,6.0,27.0,17.39,89.0
3,Queanbeyan West,NaN,NaN,NaN,NaN,675.0,NaN,6.29,1.0,10.0,0.00,46.0
4,Jerrabomberra,1185000.0,3.39,19.39,8.56,770.0,3.37,1.31,5.0,41.0,0.00,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2716,South Burnie,450000.0,0.00,4.89,NaN,NaN,NaN,NaN,0.0,4.0,-33.33,NaN
2717,South Spreyton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,-66.67,NaN
2718,East Palmerston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,100.00,NaN
2719,East Russell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,-50.00,NaN


In [24]:
df_houses.columns

Index(['Suburb', 'Median listing price',
       'Median price change - last quarter (%)',
       'Median price change - 1 year (%)', 'Median price change - 2 years (%)',
       'Median weekly rent', 'Median yield %',
       'Median rent change - 1 year (%)', 'Previous month sales',
       'Stock on market previous month', 'Stock variance vs. last year (%)',
       'Average days on market'],
      dtype='object')

In [25]:
df_town_houses.columns

Index(['Suburb', 'Median listing price',
       'Median price change - last quarter (%)',
       'Median price change - 1 year (%)', 'Median price change - 2 years (%)',
       'Median weekly rent', 'Median yield %',
       'Median rent change - 1 year (%)'],
      dtype='object')

In [27]:
df_units.columns

Index(['Suburb', 'Median listing price',
       'Median price change - last quarter (%)',
       'Median price change - 1 year (%)', 'Median price change - 2 years (%)',
       'Median weekly rent', 'Median yield %',
       'Median rent change - 1 year (%)', 'Previous month sales',
       'Stock on market previous month', 'Stock variance vs. last year (%)',
       'Average days on market'],
      dtype='object')

In [22]:
houses_vic = merge_suburb(df_units, "Victoria")
px.scatter(
    houses_vic, 
    x="Median listing price", 
    #y="Median weekly rent",
    y="Median yield %",
    hover_data={"Suburb": True},
    template="seaborn"
)

# Choropleth

In [ ]:
gdf = gpd.read_file("C:/Users/yeh/Documents/property_market_analysis/data/SA2_SHP/SA2_2021_AUST_GDA2020.shp")[[
    "SA2_CODE21", "SA2_NAME21", "geometry" 
]]

In [ ]:
geojson_data = gdf.to_crs(4326)
geojson_data.to_file('geojson_data.geojson', driver='GeoJSON') #SHP to GeoJSON
geojson = json.load(open('geojson_data.geojson', 'r'))

In [ ]:
gdf_houses_vic = houses_vic.merge(gdf.reset_index(), left_on="Suburb", right_on="SA2_NAME21")

In [ ]:
# # Create the choropleth map
# fig = px.choropleth(
#     gdf_houses_vic, 
#     geojson=geojson, 
#     locations="index", 
#     color='Median listing price',
#     hover_name='SA2_NAME21', 
#     hover_data=['Median listing price']
# )

# # Update map layout to make it more presentable
# #fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(title="Victoria")

# # Show the plot
# fig.show()